In [ ]:
import ipywidgets as ipw

from wizard import WizardApp
from structures import StructureUploadComboWidget
from relax import RelaxSubmitWidget
from bands import ComputeBandsSubmitWidget

from aiida.orm import StructureData


# Create the application steps
structure_selection_step = StructureUploadComboWidget(
    examples=[
        ('Diamond', 'miscellaneous/structures/diamond.cif'),
        ('Gallium arsenide', 'miscellaneous/structures/GaAs.xyz'),
        ('Silicon', 'miscellaneous/structures/Si.xyz'),
        ('Silicon oxide', 'miscellaneous/structures/SiO2.xyz'),
    ],
    viewer=True)
relax_step = RelaxSubmitWidget(allow_skip=True, has_next=True)
compute_bands_step = ComputeBandsSubmitWidget()

# Link the application steps
ipw.dlink((structure_selection_step, 'confirmed_structure'), (relax_step, 'input_structure'))
ipw.dlink((relax_step, 'output_structure'), (compute_bands_step, 'input_structure'))

# Propagate the configuration from the relax step to the compute band gaps step.
# TODO: uncomment when https://github.com/aiidalab/aiidalab-widgets-base/issues/60 is fixed.
#ipw.dlink((relax_step.code_group, 'selected_code'), (compute_bands_step.code_group, 'selected_code'))
ipw.dlink((relax_step.pseudo_family, 'value'), (compute_bands_step.pseudo_family, 'value'))
ipw.dlink((relax_step.number_of_nodes, 'value'), (compute_bands_step.number_of_nodes, 'value'))
ipw.dlink((relax_step.cpus_per_node, 'value'), (compute_bands_step.cpus_per_node, 'value'))

# Add the application steps to the application
app = WizardApp(
    steps=[
        ('Select structure', structure_selection_step),
        ('Relax', relax_step),
        ('Compute bands', compute_bands_step)])
relax_step.callbacks.append(lambda _: app._update_titles())
compute_bands_step.callbacks.append(lambda _: app._update_titles())

display(app)